### 1. **Setting Up the Environment**
Make sure you have the required Python packages installed. You can install them by running:

```bash
pip install torch transformers autocuda viennarna sklearn
```

We'll also be using the pre-trained `OmniGenome` model from Hugging Face for token classification. The specific model, `anonymous8/OmniGenome-186M`, is trained on RNA secondary structure prediction tasks.

### 2. **Loading the Model and Tokenizer**
The first step is to load the pre-trained model and tokenizer for RNA secondary structure prediction.

In [ ]:
import torch
import autocuda
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load the pre-trained model for secondary structure prediction
ssp_model = AutoModelForTokenClassification.from_pretrained(
    "anonymous8/OmniGenome-186M", trust_remote_code=True
).to(autocuda.auto_cuda())

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("anonymous8/OmniGenome-186M")


### 3. **Defining the Prediction Function**
Here, we define a function `zero_shot_secondary_structure_prediction` that takes a model and an RNA sequence as input and outputs the predicted secondary structure.

This function works by tokenizing the input sequence, passing it through the model, and converting the predicted tokens into secondary structure labels.

In [ ]:
device=autocuda.auto_cuda()

def zero_shot_secondary_structure_prediction(model, sequence):
    model.eval()
    inputs = tokenizer(
        sequence, return_tensors="pt", padding="max_length", truncation=True
    ).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs['logits'].argmax(dim=-1)[:, 1:-1]  # Skip special tokens
    structure = [
        model.config.id2label[prediction.item()] for prediction in predictions[0]
    ]
    return "".join(structure)

### 4. **Predicting RNA Secondary Structure**
Let's now use the function we defined to predict the secondary structure of an example RNA sequence. The sequence we're using is:

`GAAAAAAAAGGGGAGAAAUCCCGCCCGAAAGGGCGCCCAAAGGGC`


In [ ]:
# Example RNA sequence
sequence = "GAAAAAAAAGGGGAGAAAUCCCGCCCGAAAGGGCGCCCAAAGGGC"

# Predict the secondary structure
structure = zero_shot_secondary_structure_prediction(ssp_model, sequence)

# The predicted structure should look something like this:
print("Predicted structure:", structure)
# Expected output: ..........((((....))))((((....))))((((...))))

### 5. **Comparing with ViennaRNA**
For comparison, you can also use the popular RNA secondary structure prediction tool `ViennaRNA`. Below is an example of how to predict the structure of the same sequence using `ViennaRNA`.

You can install `ViennaRNA` by running:
```bash
pip install viennarna
```
Then, use the following code to predict the structure.

In [ ]:
# Uncomment the following lines to use ViennaRNA
# import ViennaRNA
# print("ViennaRNA prediction:", ViennaRNA.fold(sequence)[0])
# Expected output: ..........((((....))))((((....))))((((...))))